In [1]:
import json
from tqdm import tqdm
import os, re
import numpy as np
import pandas as pd

In [2]:
def delete_tag(s):
    s = re.sub('\{IMG:.?.?.?\}', '', s)  # 图片
    s = re.sub(re.compile(r'[a-zA-Z]+://[^\s]+'), '', s)  # 网址
    s = re.sub(re.compile('<.*?>'), '', s)  # 网页标签
    s = re.sub(re.compile('&[a-zA-Z]+;?'), ' ', s)  # 网页标签
    s = re.sub(re.compile('[a-zA-Z0-9]*[./]+[a-zA-Z0-9./]+[a-zA-Z0-9./]*'), ' ', s)
    s = re.sub("\?{2,}", "", s)
    s = re.sub("\r", "", s)
    s = re.sub("\n", ",", s)
    s = re.sub("\t", ",", s)
    s = re.sub("（", ",", s)
    s = re.sub("）", ",", s)
    s = re.sub("\u3000", "", s)
    s = re.sub(" ", "", s)
    r4 = re.compile('\d{4}[-/]\d{2}[-/]\d{2}')  # 日期
    s = re.sub(r4, '某时', s)
    return s


In [3]:
def cut_sentences(content):
    # 结束符号，包含中文和英文的
    end_flag = ['。', ';', '；']

    content_len = len(content)
    sentences = []
    tmp_char = ''
    for idx, char in enumerate(content):
        # 拼接字符
        tmp_char += char

        # 判断是否已经到了最后一位
        if (idx + 1) == content_len:
            sentences.append(tmp_char)
            break

        # 判断此字符是否为结束符号
        if char in end_flag:
            # 再判断下一个字符是否为结束符号，如果不是结束符号，则切分句子
            next_idx = idx + 1
            if not content[next_idx] in end_flag:
                sentences.append(tmp_char)
                tmp_char = ''

    return sentences

In [4]:
def metl_data(df):
    z = df.groupby(['uid'])['content'].apply(lambda x: np.concatenate(list(x))).reset_index()
    i = pd.concat([pd.Series(row['uid'], row['content']) for _, row in z.iterrows()]).reset_index()
    i.columns = ['content', 'uid']
    return i


In [5]:
def get_data(train, test):

    train = train[~train.content_type.isnull()].drop_duplicates().reset_index(drop=True)
    train['content'] = train['content'].apply(lambda x: cut_sentences(x))
    train['content'] = list(map(lambda x, y: [i for i in x if y in i], train['content'], train['entity']))
    train_n = metl_data(train)
    train = train_n.merge(train[['uid', 'entity']], how='left')
    train['content'] = train['content'].apply(lambda x: delete_tag(x))
    train['content'] = list(map(lambda x, y: x[x.find(y) - 200:x.find(y) + 200], train['content'], train['entity']))
    
    test['content'] = test['content'].apply(lambda x: delete_tag(x))
    test['content'] = test['content'].apply(lambda x: cut_sentences(x))
    test = metl_data(test)
    return train, test



In [6]:
train = pd.read_csv('../ccks2020Data/event_entity_train_data_label.csv', sep='\t', header=None,
                        names=['uid', 'content', 'content_type', 'entity'])
test = pd.read_csv('../ccks2020Data/event_entity_dev_data.csv', header=None)
test = test[0].apply(lambda x: x.split('\t')).values.tolist()
test = pd.DataFrame(test, columns=['uid', 'content'])

In [7]:
train = train[~train.content_type.isnull()].drop_duplicates().reset_index(drop=True)
train

,uid,content,content_type,entity
0,2444634,世联君汇预计2017年净利下滑近8成至853万元中超电缆(002471)再遭中超集团减持5%股份,业绩下滑,世联君汇
1,2836026,LG空调亏损严重或效仿新科 两大缺陷遭退市尴尬华兰生物(002007)三季度净利下降45% ...,业绩下滑,华兰生物
2,2809128,四方达(300179)股东减持60万股 套现414.6万元收到欧盟打款后希腊总理宣布辞职再选...,业绩下滑,巨力索具
3,2221860,单元式空调抽查不合格名单春兰、日立产品入列北大荒(600598)计提坏账净利骤降83.5%好...,业绩下滑,北大荒
4,2091205,深航副总裁闪电辞职 “股权之争”仍迷离(IPO路演)龙大肉食(002726)一季度净利“反常...,业绩下滑,龙大肉食
...,...,...,...,...
41062,2943059,/rp>YABOVIP04发布时间：2019-09-0617:11:45来源：yabocom...,财务信息造假,康美药业
41063,2469473,/rp>YABOVIP04发布时间：2019-09-0617:11:45来源：yabocom...,财务信息造假,康得新
41064,2331238,康美在我们银行贷款逾期20亿，听说下一步加星戴帽，重大财务造假退不退市就看政府出,财务信息造假,康美
41065,2926522,人家收入是事实，所以康美能涨停，再说康美只是st，又不带星，不要拿北讯和康美比，北讯是亏的不...,财务信息造假,康美


In [8]:
train['content'] = train['content'].apply(lambda x: cut_sentences(x))
train

,uid,content,content_type,entity
0,2444634,[世联君汇预计2017年净利下滑近8成至853万元中超电缆(002471)再遭中超集团减持5...,业绩下滑,世联君汇
1,2836026,[LG空调亏损严重或效仿新科 两大缺陷遭退市尴尬华兰生物(002007)三季度净利下降45%...,业绩下滑,华兰生物
2,2809128,[四方达(300179)股东减持60万股 套现414.6万元收到欧盟打款后希腊总理宣布辞职再...,业绩下滑,巨力索具
3,2221860,[单元式空调抽查不合格名单春兰、日立产品入列北大荒(600598)计提坏账净利骤降83.5%...,业绩下滑,北大荒
4,2091205,[深航副总裁闪电辞职 “股权之争”仍迷离(IPO路演)龙大肉食(002726)一季度净利“反...,业绩下滑,龙大肉食
...,...,...,...,...
41062,2943059,[/rp>YABOVIP04发布时间：2019-09-0617:11:45来源：yaboco...,财务信息造假,康美药业
41063,2469473,[/rp>YABOVIP04发布时间：2019-09-0617:11:45来源：yaboco...,财务信息造假,康得新
41064,2331238,[康美在我们银行贷款逾期20亿，听说下一步加星戴帽，重大财务造假退不退市就看政府出],财务信息造假,康美
41065,2926522,[人家收入是事实，所以康美能涨停，再说康美只是st，又不带星，不要拿北讯和康美比，北讯是亏的...,财务信息造假,康美


In [9]:
train['content'] = list(map(lambda x, y: [i for i in x if y in i], train['content'], train['entity']))
train

,uid,content,content_type,entity
0,2444634,[世联君汇预计2017年净利下滑近8成至853万元中超电缆(002471)再遭中超集团减持5...,业绩下滑,世联君汇
1,2836026,[LG空调亏损严重或效仿新科 两大缺陷遭退市尴尬华兰生物(002007)三季度净利下降45%...,业绩下滑,华兰生物
2,2809128,[四方达(300179)股东减持60万股 套现414.6万元收到欧盟打款后希腊总理宣布辞职再...,业绩下滑,巨力索具
3,2221860,[单元式空调抽查不合格名单春兰、日立产品入列北大荒(600598)计提坏账净利骤降83.5%...,业绩下滑,北大荒
4,2091205,[深航副总裁闪电辞职 “股权之争”仍迷离(IPO路演)龙大肉食(002726)一季度净利“反...,业绩下滑,龙大肉食
...,...,...,...,...
41062,2943059,[/rp>YABOVIP04发布时间：2019-09-0617:11:45来源：yaboco...,财务信息造假,康美药业
41063,2469473,[/rp>YABOVIP04发布时间：2019-09-0617:11:45来源：yaboco...,财务信息造假,康得新
41064,2331238,[康美在我们银行贷款逾期20亿，听说下一步加星戴帽，重大财务造假退不退市就看政府出],财务信息造假,康美
41065,2926522,[人家收入是事实，所以康美能涨停，再说康美只是st，又不带星，不要拿北讯和康美比，北讯是亏的...,财务信息造假,康美


In [11]:
train['content_'] = train[['content', 'entity']].apply(lambda x: [i for i in x[0] if x[1] in i])


TypeError: 'in <string>' requires string as left operand, not list

In [ ]:
train = train_n.merge(train[['uid', 'entity']], how='left')

In [ ]:
train